# MedNIST dataset using DenseNet Transfer Learning¶

In [1]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms
from torchvision.models import densenet121
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

In [2]:
from fedbiomed.researcher.requests import Requests
req  = Requests()
req.list()

2024-01-24 16:56:40,010 fedbiomed INFO - Starting researcher service...
2024-01-24 16:56:40,011 fedbiomed INFO - Waiting 3s for nodes to connect...
2024-01-24 16:56:40,379 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks
2024-01-24 16:56:43,024 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks


{'NODE_7842724a-cafa-49cc-862d-149288bbbb22': [{'name': 'Cancer image labels',
   'data_type': 'csv',
   'tags': ['cancer'],
   'description': 'This dataset keeps the img path for all classes of cancer images',
   'shape': [4600, 4],
   'dataset_id': 'dataset_eefc95a8-dd37-476c-90d0-0ee33bcee8c9',
   'dataset_parameters': None},
  {'name': '',
   'data_type': 'flamby',
   'tags': ['ixi'],
   'description': '',
   'shape': [249, 1, 48, 60, 48],
   'dataset_id': 'dataset_557acc1b-481a-4cfa-8b84-bc74f69f2ce5',
   'dataset_parameters': None,
   'dlp_id': 'dlp_29d946c8-e6e5-43a6-b4e0-a4089f7c8609'},
  {'name': 'MEDNIST',
   'data_type': 'mednist',
   'tags': ['#MEDNIST', '#dataset'],
   'description': 'MEDNIST dataset',
   'shape': [1000, 3, 64, 64],
   'dataset_id': 'dataset_7170504c-a818-441d-bab4-40dc5d13be65',
   'dataset_parameters': None}]}

## Change only classification's layer 

In [3]:
class MyTrainingPlan2(TorchTrainingPlan):

    def init_model(self, model_args):
        # Load the pre-trained DenseNet model
        model = models.densenet121(pretrained=True)
        #model = densenet121(pretrained=True)
        #model = models.densenet121(pretrained=True, weights= 'imagenet')
        # remove the classification layer 
        for param in model.features[:-1].parameters():
            param.requires_grad = False
        
        num_classes = model_args['num_classes'] # Change this to the number of classes in your dataset
        num_ftrs = model.classifier.in_features
        model.classifier= nn.Sequential(
            #nn.Linear(model[-1].in_features, 1024),
            nn.Linear(num_ftrs, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(500, num_classes)
        )
      
        return model

    def init_dependencies(self):
        return [
            "import pandas as pd",
            "from PIL import Image",
            "from torchvision import datasets, transforms, models",
            "from torch.utils.data import Dataset",
            "import torch.optim as optim",
            "from torchvision.models import densenet121"
        ]


    def init_optimizer(self, optimizer_args):        
        return optim.Adam(self.model().parameters(), lr=optimizer_args["lr"])

    # training data
    

    def training_data(self):
        # Custom torch Dataloader for MedNIST data
        print("dataset path",self.dataset_path)
        preprocess = transforms.Compose([
                transforms.Resize((224,224)),  
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomRotation(30),
                transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
           ])
    
        train_data = datasets.ImageFolder(self.dataset_path,transform = preprocess)
        train_kwargs = { 'shuffle': True}
        return DataManager(dataset=train_data, **train_kwargs)

    def training_step(self, data, target):
        output = self.model().forward(data)
        loss_func = nn.CrossEntropyLoss()
        loss   = loss_func(output, target)
        return loss




In [4]:
training_args = {
    'loader_args': { 'batch_size': 32, }, 
    'optimizer_args': {'lr': 1e-4}, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

model_args = {
    'num_classes': 6
}

In [5]:


tags =  ['#MEDNIST', '#dataset']
rounds = 1

exp = Experiment(tags=tags,
                 training_plan_class=MyTrainingPlan2,
                 model_args=model_args,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage())

from fedbiomed.common.metrics import MetricTypes
exp.set_test_ratio(.1)
exp.set_test_on_local_updates(True)
exp.set_test_metric(MetricTypes.ACCURACY)

exp.set_tensorboard(True)


2024-01-24 16:56:51,467 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks
2024-01-24 16:56:51,468 fedbiomed INFO - Node selected for training -> NODE_7842724a-cafa-49cc-862d-149288bbbb22
2024-01-24 16:56:51,470 fedbiomed DEBUG - Model file has been saved: /home/ebirgy/development/fedbiomed_github/fedbiomed/var/experiments/Experiment_0038/model_10b50a56-bb4a-456c-8fb6-0a7f5e584c98.py
Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=Dens

True

In [6]:
from fedbiomed.researcher.environ import environ
tensorboard_dir = environ['TENSORBOARD_RESULTS_DIR']

In [8]:
exp.run()

2024-01-24 16:57:49,812 fedbiomed INFO - Sampled nodes in round 0 ['NODE_7842724a-cafa-49cc-862d-149288bbbb22']
2024-01-24 16:57:49,826 fedbiomed INFO - Sending request 
					 To: NODE_7842724a-cafa-49cc-862d-149288bbbb22 
					 Request: : TRAIN
 -----------------------------------------------------------------
2024-01-24 16:57:50,179 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks
2024-01-24 16:57:52,162 fedbiomed ERROR - Training failed for node NODE_7842724a-cafa-49cc-862d-149288bbbb22: Cannot train model in round: RuntimeError('mat1 and mat2 shapes cannot be multiplied (32x1024 and 500x6)')
2024-01-24 16:57:52,163 fedbiomed ERROR - FB408: training failed on node or node did not answer during training (node = NODE_7842724a-cafa-49cc-862d-149288bbbb22)
2024-01-24 16:57:52,163 fedbiomed CRITICAL - FB407: list of nodes became empty when training (all nodes failed training or did not answer)



--------------------
Fed-BioMed researcher stopped due to exception:
FB407: list of nodes became empty when training (all nodes failed training or did not answer)
--------------------


FedbiomedSilentTerminationError: 

## Training of last model's layers

In [3]:
class MyTrainingPlan3(TorchTrainingPlan):

    def init_model(self, model_args):
        model = models.densenet121(pretrained=True)
        # For example, let's freeze all layers up to the second-to-last dense block
        for param in model.features[:-3].parameters():
            param.requires_grad = False

        # Modify the classifier to match the number of classes in your dataset
        num_ftrs = model.classifier.in_features
        num_classes = model_args['num_classes'] 
        model.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes)
           
            )
        
        return model

    def init_dependencies(self):
        return [
            "import pandas as pd",
            "from PIL import Image",
            "from torchvision import datasets, transforms, models",
            "from torch.utils.data import Dataset",
            "import torch.optim as optim"
        ]


    def init_optimizer(self, optimizer_args):        
        return optim.Adam(self.model().parameters(), lr=optimizer_args["lr"])

    # training data
    class CustomDatasetFedBioMed(Dataset):
        def __init__(self, dataset_path: str, transform=None):
            self.dataframe = pd.read_csv(dataset_path, index_col=0)
            
            self.transform = transforms.Compose([
                transforms.Resize((224,224)),  
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomRotation(30),
                transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
           ])
    
        def __len__(self):
            return len(self.dataframe)
    
        def __getitem__(self, idx):
            
            img_path = self.dataframe.iloc[idx, 0]  # the file path is in the first column
            print(img_path)
            image = Image.open(img_path).convert("RGB")
    
            label = self.dataframe.iloc[idx, 1]  # the class label is in the second column
    
            if self.transform:
                image = self.transform(image)
    
            label = torch.tensor(label, dtype=torch.long)  # if it returns a tuple instead of a tensor
    
            return image, label 

    def training_data(self):
        dataset = self.CustomDatasetFedBioMed(self.dataset_path)
        return DataManager(dataset)

    def training_step(self, data, target):
        output = self.model().forward(data)
        loss_func = nn.CrossEntropyLoss()
        loss   = loss_func(output, target)
        return loss




In [5]:
training_args = {
    'loader_args': { 'batch_size': 32, }, 
    'optimizer_args': {'lr': 1e-4}, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}
model_args={
    'num_classes': 6
}
tags =  ['#MEDNIST', '#dataset']
rounds = 1

exp = Experiment(tags=tags,
                 training_plan_class=MyTrainingPlan3,
                 model_args=model_args,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage())

from fedbiomed.common.metrics import MetricTypes
exp.set_test_ratio(.1)
exp.set_test_on_local_updates(True)
exp.set_test_metric(MetricTypes.ACCURACY)

exp.set_tensorboard(True)
    

2024-01-24 15:21:07,512 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks
2024-01-24 15:21:07,516 fedbiomed INFO - Node selected for training -> NODE_7842724a-cafa-49cc-862d-149288bbbb22
2024-01-24 15:21:07,519 fedbiomed DEBUG - Model file has been saved: /home/ebirgy/development/fedbiomed_github/fedbiomed/var/experiments/Experiment_0037/model_794ffabf-ad57-413e-a534-8895f3f463ed.py
Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=Dens

True

2024-01-24 15:21:13,859 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_7842724a-cafa-49cc-862d-149288bbbb22 
					 Round 1 Epoch: 1 | Iteration: 30/100 (30%) | Samples: 960/3200
 					 Loss: 1.039147 
					 ---------
2024-01-24 15:21:31,317 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_7842724a-cafa-49cc-862d-149288bbbb22 
					 Round 1 Epoch: 1 | Iteration: 40/100 (40%) | Samples: 1280/3200
 					 Loss: 1.039217 
					 ---------
2024-01-24 15:21:49,741 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_7842724a-cafa-49cc-862d-149288bbbb22 
					 Round 1 Epoch: 1 | Iteration: 50/100 (50%) | Samples: 1600/3200
 					 Loss: 0.717884 
					 ---------
2024-01-24 15:21:57,842 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks
2024-01-24 15:22:13,723 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_7842724a-cafa-49cc-862d-149288bbbb22 
					 Round 1 Epoch: 1 | Iteration: 60/100 (60%) | Samples: 1920/3200
 					 Loss: 0.624486 
					 ---------
2024-01-24 15:22:

In [7]:
exp.run()

2024-01-24 15:37:40,005 fedbiomed INFO - Sampled nodes in round 0 ['NODE_7842724a-cafa-49cc-862d-149288bbbb22']
2024-01-24 15:37:40,016 fedbiomed INFO - Sending request 
					 To: NODE_7842724a-cafa-49cc-862d-149288bbbb22 
					 Request: : TRAIN
 -----------------------------------------------------------------
2024-01-24 15:37:40,169 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks
2024-01-24 15:37:40,359 fedbiomed ERROR - Training failed for node NODE_7842724a-cafa-49cc-862d-149288bbbb22: Can not create validation/train data: FedbiomedRoundError("FB314: Node round error, `The method `training_data` of the TrainingPlans.TorchTrainingPlan has failed: IsADirectoryError(21, 'Is a directory')")
2024-01-24 15:37:40,360 fedbiomed ERROR - FB408: training failed on node or node did not answer during training (node = NODE_7842724a-cafa-49cc-862d-149288bbbb22)
2024-01-24 15:37:40,361 fedbiomed CRITICAL - FB407: list of nodes became empty when training (all


--------------------
Fed-BioMed researcher stopped due to exception:
FB407: list of nodes became empty when training (all nodes failed training or did not answer)
--------------------


FedbiomedSilentTerminationError: 

In [6]:
from fedbiomed.researcher.environ import environ
tensorboard_dir = environ['TENSORBOARD_RESULTS_DIR']

2024-01-24 15:37:30,261 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks
2024-01-24 15:37:36,246 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks


In [8]:
#save model, results 
exp.training_plan().export_model('./training_plan3_densenet')

2024-01-24 15:38:40,174 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks
2024-01-24 15:38:48,642 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks
2024-01-24 15:39:48,646 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks
2024-01-24 15:40:48,644 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks
2024-01-24 15:41:03,270 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks
2024-01-24 15:42:03,271 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks
2024-01-24 15:42:09,478 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks
2024-01-24 15:43:09,478 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for the tasks
2024-01-24 15:43:16,589 fedbiomed DEBUG - Node: NODE_7842724a-cafa-49cc-862d-149288bbbb22 polling for th

In [ ]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1]
for r in round_data.values():
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = r['node_id'],
        rtraining = r['timing']['rtime_training'],
        ptraining = r['timing']['ptime_training'],
        rtotal = r['timing']['rtime_total']))
print('\n')
